Na obtenção dos dados, estou preocupado em garantir que haja acessibilidade padronizada aos dados dos arquivos; adequação dos tipos de dados (categóricos, ordenados, numéricos); otimização do uso da memória; consistência do conteúdo; encaminhamento de valores faltantes; compatibilidade entre dados de diversos arquivos.
Por regras de consistência, certas bases apresentam alguns atributos em que não existem dados faltantes, enquanto outros atributos podem contar com ausências. No processo de exploração da base de dados, é importante verificar quais colunas podem se encontrar nessa situação.


**Tarefas**
1. Geração do Environment


2. Obtenção e Tratamento Preliminar dos Dados
* Fazer o download dos dados
        * Verificar como inserir break
        * fazer o código agnostico a no e uf
* ~~Identificar qual o encoding~~
* ~~Verificar quais colunas utilizar~~
* ~~Retirar linhas que não correspondam a vínculos ativos~~
* ~~Indicar esparador de decimais na importação~~
* Ler arquivo de tabela direto do .7z
* Escolher o tipo das colunas para otimização de memória e tempo de resposta
    * ~~Verificar máximo e mínimo de colunas para identificar que tipo de float ou int utilizar~~
    * ~~Colunas que estão recebendo automaticamente o tipo object estão mais pesadas~~
    * ~~criar dicionário com tipos para usar no _casting_~~
    * ~~construir os tipos categóricos ordenados com CategoricalDtype~~
    * ~~algumas colunas estavam com problema para calcular média, pois alguns valores se tornaram infinitos. Isso ocorreu pois o float16 não deu conta de representar o número registrado na memória. Alterei para float64~~
    * Talvez eu deva agregar todos os dados, verificar a importância do campo tipo de estabelecimento com CNPJ e CEI, para depois decidir se devo ignorar sua importação
        * Há duas colunas tipo de estabelecimento. Preciso ver qual das duas funciona melhor
    * Verificar como fazer o relacionamento das categorias do CBO e da CNAE
* Eliminar linhas com valores nulos ou inconsistentes
    * ~~399 é o número de municípios no arquivo pr2010, o que é consistente com o número de municípios da UF~~
    * ~~Describe apresenta algumas variáveis numéricas para as quais a média é nula~~
    * ~~~Para 2010 no AC, Faixa Etária, Faixa Remun Dezem (SM) e  Faixa Remun Média (SM) tinha alguns dados nulos~~
        * ~~Atribuir variáveis categóricas ordenadas, com uma lista fechada, implica que valores fora da categoria sejam tratados como NaN. Isso aconteceu nesses casos.~~
        * talvez eu precise criar uma aplicação que confira o volume de Nulos em todos os arquivos e gere um relatório
    * Nas variáveis categóricas existem valores -1 para dados inexistentes
* Verificar qual a melhor estratégia para agregar os dados de diversos anos e ufs
    * agregar tudo em um banco de dados?
    * agregar tudo e carregar com Vaex?
    * Trabalhar com diversos dataframes e fazer loop na hora de encadear seus resultados?
* Criar o módulo e os objetos (função ou classe) com o fluxo de trabalho e a implementação padronizada da obtenção e tratamento
    * Possivelmente eu terei que utilizar argparse
    * criar função load_rais
        * Criar um exception handling para os casos em que os arquivos ou pastas não existirem
* Fazer com que o código seja agnóstico para anos e ufs
* Preciso criar uma estrutura territorial condizente com o REGIC atual
    * Lembrar que Manaus não é um arranjo populacional
    PARA FAZER AS CODIFICAÇÕES TERRITORIAIS E DE TECNOLOGIA EU POSSO UTILIZAR A FUNÇÃO CRIADA EM https://www.vooo.pro/insights/12-tecnicas-pandas-uteis-em-python-para-manipulacao-de-dados/
    
#Define uma função genérica usando a função replace
def coding(col, codeDict):
  colCoded = pd.Series(col, copy=True)
  for key, value in codeDict.items():
    colCoded.replace(key, value, inplace=True)
  return colCoded
 
#Codificando LoanStatus como Y=1, N=0:
print('Before Coding:')
print(pd.value_counts(data["Loan_Status"]))
data["Loan_Status_Coded"] = coding(data["Loan_Status"], {'N':0,'Y':1})
print('\nAfter Coding:')
print(pd.value_counts(data["Loan_Status_Coded"]))
* Preciso incorporar as tecnologias dos produtos
* Preciso associar o nível tecnológico das atividades
* Preciso associar as categorias da POTEC
    * Houve alguns casos em que não houve associação entre o cadastro de POTEC que eu criei e CBO na RAIS. Isso ocorreu para códigos que seriam considerados como "Demmais Ocupações". Inserir fillna("Demais Ocupações") e isso deu conta, por ora. Por essa razão, quando a base de dados estiver formada, preciso conferir se não houve casos de POTEC que não foi classificado.
    
    * O POTEC registrado se encontra no nível mais granular. Preciso fazer agrupamentos
    
    Existe uma forma de criar categorias com labels e códigos de interesse do desenvolvedor. https://pandas.pydata.org/pandas-docs/version/0.23.4/generated/pandas.Categorical.from_codes.html. Para que essa função funcione, é preciso garantir que não haverá NAN, mas que os valores serão substituídos por 0 https://www.classes.cs.uchicago.edu/archive/2019/winter/12200-1/labs/lab4/index.html.

* Parece que a variável df_derivado, que fiz em algumas funções, não é necessária. Ela pode ser df e segue dando certo, como fiz em analysis
    
Na primeira importação, o consumo de memória do arquivo pr2010 era de mais de 1,5Gb. Com algumas alterações realizadas (indicação de colunas a importar, associação de dtypes mais econômicos etc.), o consumo de memória passou para 0,14Gb.

In [1]:
import pandas as pd
# import modin.pandas as pd
from pandas.api.types import CategoricalDtype
import numpy as np
import os
from itertools import product

### Código de Obtenção e Tratamento

In [2]:
def load_rais(uf=None, year=None, treating=True):
    '''
    Loads Rais data into Pandas DataFrame regarding the year and Brazilian
    federation unity (UF) selected. If treating is set to True (default), 
    loads a predefined set of columns, overrides the automatically 
    inputed data types with others selected to enhance performance, 
    besides deleting registers of non active jobs in 12/31 of the selected year.
    
    '''
    if uf==None or year==None:
        print('Please set both uf and year.')
    
    else:    
    
        if treating == False:
            df = pd.read_table(
                f'data/rais_original//{year}/{uf.upper()}{year}.txt'
                , encoding='Latin-1'
                , sep=';'
                , decimal=','
                )

        else:

            list_column_keep_2010 = [
                'CBO Ocupação 2002'
                , 'CNAE 2.0 Classe'
                , 'CNAE 95 Classe'
                , 'Vínculo Ativo 31/12'
                , 'Faixa Etária'
                , 'Faixa Remun Dezem (SM)'
                , 'Faixa Remun Média (SM)'
                , 'Faixa Tempo Emprego'
                , 'Escolaridade após 2005'
                , 'Mun Trab'
                , 'Município'
                , 'Nacionalidade'
                , 'Natureza Jurídica'
                , 'Vl Remun Dezembro Nom'
                , 'Vl Remun Dezembro (SM)'
                , 'Vl Remun Média Nom'
                , 'Vl Remun Média (SM)'
                , 'CNAE 2.0 Subclasse'
                , 'Tamanho Estabelecimento'
                , 'Tipo Estab'
                , 'Tipo Estab.1'
                , 'Tipo Vínculo'
            ]

            type_cat_tam_estabelecimento = CategoricalDtype(categories=[ -1, 1,  2,  3,  4,  5,  6,  7,  8,  9, 10], ordered=True)
            type_cat_escolaridade = CategoricalDtype(categories=[ -1, 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11], ordered=True)
            type_cat_faixa_etaria = CategoricalDtype(categories=['01', '02', '03', '04', '05', '06', '07', '08'], ordered=True)
            type_cat_faixa_remu_media = CategoricalDtype(
                categories=[
                    '00'
                    ,'01'
                    , '02'
                    , '03'
                    , '04'
                    , '05'
                    , '06'
                    , '07'
                    , '08'
                    , '09'
                    , '10'
                    , '11'
                ]
                , ordered=True
            )

            type_cat_faixa_remu_media_dezembro = CategoricalDtype(
                categories=[
                    '00'
                    , '01'
                    , '02'
                    , '03'
                    , '04'
                    , '05'
                    , '06'
                    , '07'
                    , '08'
                    , '09'
                    , '10'
                    , '11'
                    , '12'
                ]
                , ordered=True
            )


            dic_dtype = {
                'CBO Ocupação 2002' : 'category'
                ,'CNAE 2.0 Classe' : 'category'
                ,'CNAE 95 Classe' : 'category'
                ,'Faixa Etária' : type_cat_faixa_etaria
                ,'Faixa Remun Dezem (SM)' : type_cat_faixa_remu_media_dezembro
                ,'Faixa Remun Média (SM)' : type_cat_faixa_remu_media
                ,'Faixa Tempo Emprego' : 'category'
                ,'Escolaridade após 2005' : type_cat_escolaridade
                ,'Mun Trab' : 'category'
                ,'Município' : 'category'
                ,'Nacionalidade' : 'category'
                ,'Natureza Jurídica' : 'category'
                ,'CNAE 2.0 Subclasse' : 'category'
                ,'Tamanho Estabelecimento' : type_cat_tam_estabelecimento
                ,'Tipo Estab' : 'category'
                ,'Tipo Estab.1' : 'category'
                ,'Tipo Vínculo' : 'category'
                , 'Vl Remun Dezembro Nom' : np.float64
                , 'Vl Remun Média Nom' : np.float32
                , 'Vl Remun Dezembro (SM)' : np.float64
                , 'Vl Remun Média (SM)' : np.float64
                , 'Tempo Emprego' : np.float64
                , 'Vínculo Ativo 31/12' : 'category'
            }
            
            
            

            df = pd.read_table(
                f'data/rais_original/{year}/{uf.upper()}{year}.txt'
                , encoding='Latin-1'
                , sep=';'
                , usecols=list_column_keep_2010
                , dtype=dic_dtype
                , decimal=','
            )


            df = df.loc[df['Vínculo Ativo 31/12'] == '1']
            column_drop_2010 = ['Vínculo Ativo 31/12']
            df.drop(axis=1, labels=column_drop_2010, inplace=True)
            df.reset_index(inplace=True)
            df.drop(labels='index', axis=1, inplace=True)
            
            
            
            df_territory_tese = pd.read_csv('../app_territory/data/territorio_tese.csv', sep=';'
                                , dtype={'Município':'category', 'territorio_tese': 'category'})
            df = df.merge(df_territory_tese, on='Município', how='left')
            
            
            df_territory_regic = pd.read_csv('../app_territory/data/territorio_regic_ap_2018.csv', sep=';' 
                                             , dtype={'Município':'category', 'arranjo': 'category'})
            df = df.merge(df_territory_regic, on='Município', how='left')
            
            
            
            df_sectors_technology = pd.read_csv(
                '../app_rais/data/sector_structure/estrutura setorial.csv', 
                sep=';', 
                dtype={
                    'cod_cnae_grupo':'category'
                    ,'CNAE_grupo':'category'
                    , 'technology_industries':'category'
                }) 
            dict_services_knowledge = {row[1]['cod_cnae_grupo']:row[1]['knowledge_services'] for row in df_sectors_technology.iterrows()}
            df['knowledge_services'] = df.loc[:, 'CNAE 2.0 Classe'].str.slice(0,3).map(dict_services_knowledge)
            
            dict_industries_tec = {row[1]['cod_cnae_grupo']:row[1]['technology_industries'] for row in df_sectors_technology.iterrows()}
            df['technology_industries'] = df.loc[:, 'CNAE 2.0 Classe'].str.slice(0,3).map(dict_industries_tec)
            
            
            df_potec = pd.read_csv(
                '../app_rais/data/occupation_structure/potec.csv'
                , sep=';'
                , dtype={
                    'CBO Ocupação 2002':'category'
                    ,'potec':'category'
                })
            dict_potec = {row[1]['CBO Ocupação 2002']:row[1]['potec'] for row in df_potec.iterrows()}
            df['potec'] = df.loc[:, 'CBO Ocupação 2002'].map(dict_potec).fillna('Demais Ocupações')
                
        
    return df

In [3]:
def save_treated(df=None, year=None, uf=None, compression_method='zip', encoding='utf-8'):
    
    
    if year == None or uf == None:
        print('Please do not forget to insert df, year and uf.')
    
    else:
        
        # inserir aqui uma condição para verificar se o arquivo já existe.
        
        directory = os.path.join(os.getcwd(), 'data\\rais_treated\\', str(year) + '\\')
        os.makedirs(directory, exist_ok=True)
        
        
        compression_opts = dict(method=compression_method,
                        archive_name=f'{uf.upper()}{year}.csv')
    
        df.to_csv(
            f'data/rais_treated/{year}/{uf.upper()}{year}.zip', 
            compression=compression_opts
            , encoding=encoding
            , sep=';'
            , decimal=','
            , index=False
        )

In [4]:
def original_to_treated_loop(ufs=None, year=None):

    for uf, year  in product(ufs, year):

        if year == '2018': # Não está funcionando propriamente
            break

        print(uf.upper(), year)


        df = load_rais(uf=uf, year=year, treating=True)

        save_treated(df=df, year=year, uf=uf)
    

In [ ]:
original_to_treated_loop(ufs=['pe', 'rs', 'pr'], year=[year for year in range(2007, 2018)])

PE 2007
PE 2008
PE 2009
PE 2010
PE 2011
PE 2012
PE 2013
PE 2014
PE 2015
PE 2016
PE 2017
RS 2007
RS 2008
RS 2009
RS 2010
RS 2011
RS 2012
RS 2013
RS 2014
RS 2015
RS 2016
RS 2017
PR 2007
